In [ ]:
'''clear to run, check with nate if issues'''


#import pandas for data processing
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
def fname_ls_builder(fin_path,initial=True):
  #Find all files in data directory
  from os import listdir
  from os.path import isfile, join
  if initial:
    return [f for f in listdir(fin_path) if isfile(join(fin_path,f)) and (not ('_ds' in f or '_r' in f))]
  else:
    return [f for f in listdir(fin_path) if (isfile(join(fin_path,f)) and ('_ds' in f or '_r' in f))]

def df_builder(fin_path,fname_ls):
  #this function imports data from CSV files to one large dataframe

  #create list to hold dataframes
  df_ls=[]

  #read in each file
  for i in fname_ls:
    temp_df=pd.read_csv(fin_path+i,index_col=0)
    #print(i,'loaded\n\n')
    df_ls.append(temp_df)

  #create one large df
  df=pd.concat(df_ls)
  #print(df.shape)
  return df

def split_data(X):
  #separate y from X
  y=X['label']
  X.drop(['label'],axis=1,inplace=True)
  #split into train and dev sets
  #print(X,'\n\n',y)
  return X,y


#set filepaths for import/export
pkin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Labeled/'

pk_fname_ls=fname_ls_builder(pkin_path)

pk_df=df_builder(pkin_path,pk_fname_ls)

#drop relative intensities
pk_df.drop(columns=[i for i in pk_df.columns.values if 'val' in i],inplace=True)

pk_df,y=split_data(pk_df)


In [ ]:
#split into train and dev sets
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(pk_df, y, test_size = 0.20, random_state = 0)
#print(X_train,'\n\n',X_dev,'\n\n',y_train,'\n\n',y_dev)

#build a df of synthetic data for the training set
pk_fname_ls2=fname_ls_builder(pkin_path,initial=False)
pk_df2=df_builder(pkin_path,pk_fname_ls2)

#create columns to compare similar samples
temp_pk=pk_df2.copy()
X_temp=X_train.copy()
temp_pk.index.name='temp'
X_temp.index.name='temp'
#print('test X_train', X_train,'\n\n\n\n')
temp_pk.reset_index(inplace=True)
X_temp.reset_index(inplace=True)
temp_pk.rename(columns={temp_pk.index.name:'temp'},inplace=True)
#print(X_temp,'\n\n',temp_pk,'\n\nX_train\n\n',X_train)

#print(X_temp['temp'].str.split('-',expand=True)[[0,1]])
X_temp[['file','line']]=X_temp['temp'].str.split('-',expand=True)[[0,1]]
temp_pk[['file','line']]=temp_pk['temp'].str.split('-',expand=True)[[0,1]]
for i in X_temp.index.values:
#for i in range(10):
  #print(X_temp.loc[i,['temp']])
  temp_df=temp_pk.loc[temp_pk['line']==X_temp.loc[i,['line']].values[0]]
  #print('temp should be equal to string above\n',temp_df)
  temp_df=temp_df.loc[temp_df['file'].str.contains(X_temp.loc[i,['file']].values[0])]
  
  temp_df.set_index(keys=temp_df['temp'],drop=True,inplace=True)
  temp_df,temp_y=split_data(temp_df.set_index(keys=temp_df.temp,drop=True).drop(columns=['file','line','temp']))
  X_train.append(temp_df)
  #print(X_train)
  y_train.append(temp_y)
  if i%1000==0:
    print(i)

print(X_train,'\n\n',y_train)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
                          peak1  peak2  peak3  peak4       peak5
hb_train_015s_2500-1758     680    225    365  545.0  430.000000
qtz_train_003s_1148-633     465    205    355  590.0  690.000000
qtz_train_003s_4760-3823    465    205    355  700.0  630.000000
aug_train_030s_5600-2080    670    225    370  530.0  300.000000
bt_train_120s_1225-184      680    340    555  435.0  275.000000
...                         ...    ...    ...    ...         ...
ms_train_015s_6400-2764     265    700    405  630.0  545.000000
mc_train_015s_5600-2889     355    295    505  435.0  640.000000
en_train_015s_5600-5086     345    675    410  240.0  540.000000
ms_train_015s_6400-440      265    700    410  630.0    0.557015
bt_train_060s_4701-2732     

In [ ]:
#scale data((X-mean)/std_dev)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_dev_sc = sc.fit_transform(X_dev)
#print(X.shape)

In [ ]:
from sklearn.decomposition import PCA
#run PCA
X_train_p=PCA(n_components=3).fit_transform(X_train_sc)
X_dev_p=PCA(n_components=3).fit_transform(X_dev_sc)
#print(X_transformed)

In [ ]:
#create model
from sklearn.svm import LinearSVC
model=LinearSVC(max_iter=5000).fit(X_train_sc,y_train)
modelp=LinearSVC(max_iter=5000).fit(X_train_p,y_train)

In [ ]:
#check model
print(model.score(X_dev_sc,y_dev))
print(modelp.score(X_dev_p,y_dev))

0.798022855069835
0.6632504988209686


In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_dev,model.predict(X_dev_sc))
pcm=confusion_matrix(y_dev,modelp.predict(X_dev_p))
print(cm,'\n\n\n',pcm)

[[1842  101    0   42    0    5    0   12   14]
 [ 152 1008    2    0    0    1    0    0    0]
 [   3    0  483   49    0    9  434   16    0]
 [   0    0   17 1052    4    5   79    1    0]
 [   0    2    0    0 1245    0    0    2    0]
 [  45   31    7    6    5  916    0    5    7]
 [   0   17   89    7    0    0 1012    2    0]
 [   0    0    0    0    1    0    0 1153    0]
 [ 340  561   45   24    5   75    2    3   88]] 


 [[1785   70    0    1   11   67   76    6    0]
 [ 154 1006    0    0    0    1    2    0    0]
 [  40   24  458    0    7    8  445   12    0]
 [ 651  405   18   16   11    3   53    1    0]
 [   0    0    0    0 1245    2    0    2    0]
 [  60  115    0    7    6  702    3  129    0]
 [  27   18  131    0    1    1  949    0    0]
 [   0    0    0    0    3    0    0 1151    0]
 [ 320  571    0  167    7   76    1    0    1]]


In [ ]:
testin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Unlabeled/'
test_ls=fname_ls_builder(testin_path)
test_df=df_builder(testin_path,test_ls)
test_df.drop(columns=[i for i in test_df.columns.values if 'val' in i],inplace=True)
X_test=sc.fit_transform(test_df)
X_test_p=PCA(n_components=3).fit_transform(X_test)

saved_data=r'/content/drive/My Drive/ML Spectroscopy/Programs/Data Processing/Saved Lists/'

pd.DataFrame(data=model.decision_function(X_test),index=test_df.index.values,columns=sorted(y.unique())).to_csv(saved_data+r'linear_svc.csv')
pd.DataFrame(data=modelp.decision_function(X_test_p),index=test_df.index.values,columns=sorted(y.unique())).to_csv(saved_data+r'linear_svc_pca.csv')
